# Master Thesis - Mattia Piazzalunga
In this notebook, the generation of the Italian dataset is carried out thanks to the collaboration with ANSA.

*Tile*: Bridging a GAP: Text Style Transfer from Journalistic to Conversational for enhanced social media dissemination of news

*Supervisor*: Gabriella Pasi <br>
*Author*: Mattia Piazzalunga

*University*: Bicocca University of Milan <br>
*Department*: Informatics, Systems and Communication <br>
*Course*: Computer Science <br>
*Academic year*: 2023/2024

*Info*: This notebook was run locally. Download the whole repository before running.

*For suggestions or questions*: mattiapiazzalunga@outlook.com

## Inizializzazion

### Dowloading libraires

In [303]:
!pip install requests 

### Importing libraries

In [304]:
import glob
import json
import pandas as pd
from urllib.parse import urlparse, parse_qs, unquote
import requests
from urllib.parse import urlparse
import os
import re
import requests
from bs4 import BeautifulSoup

### Generating dataframe from json

Extracting dataframe from json and removing nan

In [305]:
# Define a class to process JSON files
class JSONFileProcessor:
    def __init__(self, directory, attributes):
        self.directory = directory
        self.attributes = attributes

    def extract_urls(self, text):
        # Define a regex pattern to match URLs
        url_pattern = r'https?://[^\s]+'
        # Find all URLs in the text (0 or more)
        urls = re.findall(url_pattern, text)
        # Remove the URLs from the text
        content_without_urls = re.sub(url_pattern, '', text)
        return urls, content_without_urls

    def get_content_list(self):
        # Get list of all JSON files in the specified directory
        all_files = glob.glob(os.path.join(self.directory, "*.json"))

        if not all_files:
            raise ValueError(f"No JSON files found in directory: {self.directory}")

        li = []

        # Print the files being processed (debugging step)
        print(f"Files found: {all_files}")

        # Iterate over all files
        for filepath in all_files:
            try:
                # Load each JSON file with UTF-8-sig encoding to handle BOM
                with open(filepath, 'r', encoding='utf-8-sig') as file:
                    data = json.load(file)

                # Iterate over the data and extract relevant fields
                for item in data:
                    try:
                        # Check if the item has all required attributes
                        if all(attr in item for attr in self.attributes):
                            # Create a dictionary for the current item
                            record = {attr: item[attr] for attr in self.attributes}

                            # Add the 'filename' field with the base name of the file, and place it first
                            record = {'filename': os.path.basename(filepath).replace('.json', ''), **record}

                            # Extract URLs from the content field and remove URLs from the content
                            if 'content' in record:
                                urls, cleaned_content = self.extract_urls(record['content'])
                                record['content'] = cleaned_content.strip()  # Update the content without URLs
                                record['link'] = urls  # Store the list of URLs in the 'link' column
                                record['url_count'] = len(urls)  # Store the number of URLs in 'url_count' column
                            else:
                                record['link'] = []
                                record['url_count'] = 0

                            # Append the record as a DataFrame
                            li.append(pd.DataFrame([record]))
                        else:
                            print(f"Skipping record in file {filepath} due to missing attributes: {item}")
                    except Exception as e:
                        print(f"Error processing record in file {filepath}: {e}")
            except Exception as e:
                print(f"Error reading file {filepath}: {e}")

        # If no valid data was collected, raise an error
        if not li:
            raise ValueError("No valid data could be extracted from the JSON files.")

        # Concatenate all DataFrames into a single DataFrame
        frame = pd.concat(li, axis=0, ignore_index=True)

        # Return the DataFrame
        return frame

In [306]:
# Load the dataframe
processor = JSONFileProcessor(directory='../starting_datasets/IT/', attributes=["content"])
df = processor.get_content_list()

Files found: ['../starting_dataset/IT\\ansa.json']


In [307]:
#Check the dataframe
df.head()

,filename,content,link,url_count
0,ansa,Un base jumper di 33 anni si è schiantato al s...,[https://l.facebook.com/l.php?u=https%3A%2F%2F...,1
1,ansa,Sterminate le api sul tetto del ministero dell...,[https://l.facebook.com/l.php?u=https%3A%2F%2F...,1
2,ansa,Le laureate italiane guadagnano la metà dei co...,[https://l.facebook.com/l.php?u=https%3A%2F%2F...,1
3,ansa,Le opposizioni sono indisponibili a rinnovare ...,[https://l.facebook.com/l.php?u=https%3A%2F%2F...,1
4,ansa,"Beyoncé 'troppo nera', snobbata ai premi del c...",[https://l.facebook.com/l.php?u=https%3A%2F%2F...,1


In [308]:
# Replace empty strings with pd.NA
df.replace("", pd.NA, inplace=True)

# Replace 'nan' and '<na>' strings with pd.NA
df.replace(['nan', '<na>'], pd.NA, inplace=True)

In [309]:
len(df)

1830

In [310]:
df_filtered = df.dropna()

In [311]:
len(df_filtered)

1827

In [312]:
# Remove rows with no URLs
df_filtered = df_filtered[df_filtered['url_count'] != 0]

In [313]:
len(df_filtered)

1802

Removing evident harmful noise

In [314]:
# Remove rows where 'content' starts with "with a story:" after trimming and converting to lowercase
df_filtered = df_filtered[~df_filtered['content'].str.strip().str.lower().str.startswith("with a story:")]

In [315]:
len(df_filtered)

1701

### Getting real urls

In [316]:
# Function to extract the real URL from a Facebook redirect link
def extract_real_url(facebook_url):
    try:
        # Parse the URL and extract the query parameters
        parsed_url = urlparse(facebook_url)
        query_params = parse_qs(parsed_url.query)

        # The real URL is contained in the 'u' parameter
        if 'u' in query_params:
            real_url = unquote(query_params['u'][0])  # Decode the URL encoded in 'u' parameter
            return real_url
        return facebook_url  # If 'u' is not present, return the original URL
    except Exception as e:
        return None  # Return None if any issue occurs

# Function to process a list of URLs and extract real URLs from Facebook redirect links
def resolve_urls(links):
    if isinstance(links, list):
        return [extract_real_url(url) for url in links]
    return []

# Apply the function to each cell in the 'link' column
df_filtered['resolved_link'] = df_filtered['link'].apply(resolve_urls)

In [317]:
df_filtered.head()

,filename,content,link,url_count,resolved_link
0,ansa,Un base jumper di 33 anni si è schiantato al s...,[https://l.facebook.com/l.php?u=https%3A%2F%2F...,1,[https://ow.ly/okhO50TkhvH]
1,ansa,Sterminate le api sul tetto del ministero dell...,[https://l.facebook.com/l.php?u=https%3A%2F%2F...,1,[https://ow.ly/TJb750TkgWX]
2,ansa,Le laureate italiane guadagnano la metà dei co...,[https://l.facebook.com/l.php?u=https%3A%2F%2F...,1,[https://ow.ly/8Fjx50TkfhA]
3,ansa,Le opposizioni sono indisponibili a rinnovare ...,[https://l.facebook.com/l.php?u=https%3A%2F%2F...,1,[https://www.ansa.it/sito/notizie/politica/202...
4,ansa,"Beyoncé 'troppo nera', snobbata ai premi del c...",[https://l.facebook.com/l.php?u=https%3A%2F%2F...,1,[https://ow.ly/H86V50TkbfB]


In [318]:
# Function to get the final URL after following redirects
def get_final_url(url):
    try:
        response = requests.get(url, allow_redirects=True, timeout=10)
        final_url = response.url
        return final_url
    except requests.RequestException:
        return url  # Return the original URL if any issue occurs

# Function to process each list of URLs, resolving only non-ansa and non-facebook URLs
def resolve_non_ansa_facebook_urls(links):
    resolved = []
    for url in links:
        parsed_url = urlparse(url)
        # Check if the URL is NOT from 'www.ansa.it' or 'www.facebook.com'
        if not (parsed_url.netloc == 'www.ansa.it' or parsed_url.netloc == 'www.facebook.com'):
            resolved.append(get_final_url(url))  # Resolve and replace non-ansa, non-facebook URLs
        else:
            resolved.append(url)  # Keep the original URL if it's from 'ansa' or 'facebook'
    return resolved

In [319]:
# Apply the function to each cell in the 'resolved_link' column
df_filtered['resolved_link'] = df_filtered['resolved_link'].apply(resolve_non_ansa_facebook_urls)

In [320]:
df_filtered.head()

,filename,content,link,url_count,resolved_link
0,ansa,Un base jumper di 33 anni si è schiantato al s...,[https://l.facebook.com/l.php?u=https%3A%2F%2F...,1,[https://www.ansa.it/lombardia/notizie/2024/09...
1,ansa,Sterminate le api sul tetto del ministero dell...,[https://l.facebook.com/l.php?u=https%3A%2F%2F...,1,[https://www.ansa.it/canale_terraegusto/notizi...
2,ansa,Le laureate italiane guadagnano la metà dei co...,[https://l.facebook.com/l.php?u=https%3A%2F%2F...,1,[https://www.ansa.it/sito/notizie/economia/pmi...
3,ansa,Le opposizioni sono indisponibili a rinnovare ...,[https://l.facebook.com/l.php?u=https%3A%2F%2F...,1,[https://www.ansa.it/sito/notizie/politica/202...
4,ansa,"Beyoncé 'troppo nera', snobbata ai premi del c...",[https://l.facebook.com/l.php?u=https%3A%2F%2F...,1,[https://www.ansa.it/canale_lifestyle/notizie/...


In [321]:
# Function to filter out URLs that don't contain "/notizie/"
def filter_notizie_links(links):
    return [url for url in links if '/notizie/' in url]

In [322]:
df_filtered['resolved_link'] = df_filtered['resolved_link'].apply(filter_notizie_links)

In [323]:
df_filtered['url_count'] = df_filtered['resolved_link'].apply(len)

In [324]:
df_filtered

,filename,content,link,url_count,resolved_link
0,ansa,Un base jumper di 33 anni si è schiantato al s...,[https://l.facebook.com/l.php?u=https%3A%2F%2F...,1,[https://www.ansa.it/lombardia/notizie/2024/09...
1,ansa,Sterminate le api sul tetto del ministero dell...,[https://l.facebook.com/l.php?u=https%3A%2F%2F...,1,[https://www.ansa.it/canale_terraegusto/notizi...
2,ansa,Le laureate italiane guadagnano la metà dei co...,[https://l.facebook.com/l.php?u=https%3A%2F%2F...,1,[https://www.ansa.it/sito/notizie/economia/pmi...
3,ansa,Le opposizioni sono indisponibili a rinnovare ...,[https://l.facebook.com/l.php?u=https%3A%2F%2F...,1,[https://www.ansa.it/sito/notizie/politica/202...
4,ansa,"Beyoncé 'troppo nera', snobbata ai premi del c...",[https://l.facebook.com/l.php?u=https%3A%2F%2F...,1,[https://www.ansa.it/canale_lifestyle/notizie/...
...,...,...,...,...,...
1825,ansa,"""Il mio pensiero va alle mamme dei soldati ucr...",[https://l.facebook.com/l.php?u=http%3A%2F%2Fo...,1,[https://www.ansa.it/sito/notizie/politica/202...
1826,ansa,Wikipedia scritta con l'intelligenza artificia...,[https://l.facebook.com/l.php?u=http%3A%2F%2Fa...,1,[https://www.ansa.it/sito/notizie/tecnologia/h...
1827,ansa,Zerocalcare ha messo i riflettori sugli ezidi ...,[https://l.facebook.com/l.php?u=https%3A%2F%2F...,1,[https://www.ansa.it/sito/notizie/cultura/libr...
1828,ansa,L'Antitrust ha avviato un'istruttoria nei conf...,[https://l.facebook.com/l.php?u=https%3A%2F%2F...,1,[https://www.ansa.it/sito/notizie/cultura/2023...


In [325]:
len(df_filtered)

1701

In [326]:
# Remove rows with no URLs
df_filtered = df_filtered[df_filtered['url_count'] != 0]

In [327]:
len(df_filtered)

1519

In [328]:
# Function to remove the all URLs keeping only the last one.
def remove_urls_not_last(links):
    if len(links) > 1:
        return [links[-1]]  # Keep only the last URL if there are multiple
    return links  # Return the original list if there's only one or no URLs

In [329]:
# Step 3: Remove the first URL if there are two URLs
df_filtered.loc[:, 'resolved_link'] = df_filtered['resolved_link'].apply(remove_urls_not_last)

In [330]:
df_filtered.head()

,filename,content,link,url_count,resolved_link
0,ansa,Un base jumper di 33 anni si è schiantato al s...,[https://l.facebook.com/l.php?u=https%3A%2F%2F...,1,[https://www.ansa.it/lombardia/notizie/2024/09...
1,ansa,Sterminate le api sul tetto del ministero dell...,[https://l.facebook.com/l.php?u=https%3A%2F%2F...,1,[https://www.ansa.it/canale_terraegusto/notizi...
2,ansa,Le laureate italiane guadagnano la metà dei co...,[https://l.facebook.com/l.php?u=https%3A%2F%2F...,1,[https://www.ansa.it/sito/notizie/economia/pmi...
3,ansa,Le opposizioni sono indisponibili a rinnovare ...,[https://l.facebook.com/l.php?u=https%3A%2F%2F...,1,[https://www.ansa.it/sito/notizie/politica/202...
4,ansa,"Beyoncé 'troppo nera', snobbata ai premi del c...",[https://l.facebook.com/l.php?u=https%3A%2F%2F...,1,[https://www.ansa.it/canale_lifestyle/notizie/...


In [331]:
# Function to extract the first (and only) URL from the list in 'resolved_link'
def extract_single_url(links):
    return links[0] if links else pd.NA

In [332]:
# Create a copy of df_filtered to avoid the SettingWithCopyWarning
df_filtered = df_filtered.copy()

# Rename 'link' column to 'link2'
df_filtered.rename(columns={'link': 'link2'}, inplace=True)

# Apply the extract_single_url function to the 'resolved_link' column and assign it to a new 'link' column
df_filtered['link'] = df_filtered['resolved_link'].apply(extract_single_url)

# Drop the unnecessary columns: 'link2', 'url_count', 'resolved_link', and 'filename'
df_filtered.drop(columns=['link2', 'url_count', 'resolved_link', 'filename'], inplace=True)


In [333]:
df_filtered.head()

,content,link
0,Un base jumper di 33 anni si è schiantato al s...,https://www.ansa.it/lombardia/notizie/2024/09/...
1,Sterminate le api sul tetto del ministero dell...,https://www.ansa.it/canale_terraegusto/notizie...
2,Le laureate italiane guadagnano la metà dei co...,https://www.ansa.it/sito/notizie/economia/pmi/...
3,Le opposizioni sono indisponibili a rinnovare ...,https://www.ansa.it/sito/notizie/politica/2024...
4,"Beyoncé 'troppo nera', snobbata ai premi del c...",https://www.ansa.it/canale_lifestyle/notizie/p...


### Scraping ANSA website

In [334]:
# Define a function to scrape the news article text from ANSA website
def scrape_ansa_article(url):
    try:
        # Request the web page
        response = requests.get(url)
        response.raise_for_status()  # Check for HTTP errors
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find the main content container (specific to ANSA website)
        article_body = soup.find('div', {'class': 'news-txt'})  # Class where ANSA stores news text

        if not article_body:
            return pd.NA  # Return NaN if article content is not found

        # Extract all paragraph text, excluding ads or unnecessary tags
        paragraphs = article_body.find_all('p')
        article_text = ' '.join([para.get_text(strip=True) for para in paragraphs])

        return article_text
    except Exception:
        return pd.NA  # Return NaN on any error

In [339]:
# Copy the first row as a DataFrame
scraped_df = df_filtered.copy(deep=True)

# Apply the scraping function to the 'url' of the first row
scraped_df['original'] = scraped_df['link'].apply(scrape_ansa_article)

In [340]:
scraped_df.head()

,content,link,original
0,Un base jumper di 33 anni si è schiantato al s...,https://www.ansa.it/lombardia/notizie/2024/09/...,Uno sportivo estremo americano di 33 anni è mo...
1,Sterminate le api sul tetto del ministero dell...,https://www.ansa.it/canale_terraegusto/notizie...,L'alveare del ministro Lollobrigida è stato at...
2,Le laureate italiane guadagnano la metà dei co...,https://www.ansa.it/sito/notizie/economia/pmi/...,"In Italia, le giovani donne con una laurea gua..."
3,Le opposizioni sono indisponibili a rinnovare ...,https://www.ansa.it/sito/notizie/politica/2024...,"""Le opposizioni sono indisponibili a rinnovare..."
4,"Beyoncé 'troppo nera', snobbata ai premi del c...",https://www.ansa.it/canale_lifestyle/notizie/p...,Beyoncé è troppo nera per i Country Music Awar...


In [341]:
#Swapping columns
scraped_df = scraped_df[['original', 'content']]

In [343]:
scraped_df.head()

,original,content
0,Uno sportivo estremo americano di 33 anni è mo...,Un base jumper di 33 anni si è schiantato al s...
1,L'alveare del ministro Lollobrigida è stato at...,Sterminate le api sul tetto del ministero dell...
2,"In Italia, le giovani donne con una laurea gua...",Le laureate italiane guadagnano la metà dei co...
3,"""Le opposizioni sono indisponibili a rinnovare...",Le opposizioni sono indisponibili a rinnovare ...
4,Beyoncé è troppo nera per i Country Music Awar...,"Beyoncé 'troppo nera', snobbata ai premi del c..."
...,...,...
1825,Papa Francesco è a Piazza San Pietro per l'udi...,"""Il mio pensiero va alle mamme dei soldati ucr..."
1826,Wikipedia scritta con l'intelligenza artificia...,Wikipedia scritta con l'intelligenza artificia...
1827,MADRID - Zerocalcare ha messo i riflettori sug...,Zerocalcare ha messo i riflettori sugli ezidi ...
1828,L'Antitrust ha avviato un'istruttoria nei conf...,L'Antitrust ha avviato un'istruttoria nei conf...


In [344]:
# Remove rows with no scraped content
scraped_df = scraped_df.dropna()

In [348]:
scraped_df

,original,content
0,Uno sportivo estremo americano di 33 anni è mo...,Un base jumper di 33 anni si è schiantato al s...
1,L'alveare del ministro Lollobrigida è stato at...,Sterminate le api sul tetto del ministero dell...
2,"In Italia, le giovani donne con una laurea gua...",Le laureate italiane guadagnano la metà dei co...
3,"""Le opposizioni sono indisponibili a rinnovare...",Le opposizioni sono indisponibili a rinnovare ...
4,Beyoncé è troppo nera per i Country Music Awar...,"Beyoncé 'troppo nera', snobbata ai premi del c..."
...,...,...
1825,Papa Francesco è a Piazza San Pietro per l'udi...,"""Il mio pensiero va alle mamme dei soldati ucr..."
1826,Wikipedia scritta con l'intelligenza artificia...,Wikipedia scritta con l'intelligenza artificia...
1827,MADRID - Zerocalcare ha messo i riflettori sug...,Zerocalcare ha messo i riflettori sugli ezidi ...
1828,L'Antitrust ha avviato un'istruttoria nei conf...,L'Antitrust ha avviato un'istruttoria nei conf...


### Removing harmful content

In [365]:
# Remove "Riproduzione\s+riservata\s+©\s+Copyright\s+ANSA" from 'original' column after trimming and converting to lowercase   
scraped_cleaned_df = scraped_df.copy()
scraped_cleaned_df['original'] = scraped_cleaned_df['original'].str.strip().str.replace(r'\s*Riproduzione\s+riservata\s+©\s+Copyright\s+ANSA\s*$', '', case=False, regex=True)

In [366]:
# Remove "with a story:" from 'content' column after trimming and converting to lowercase   
scraped_cleaned_df['content'] = scraped_cleaned_df['content'].str.strip().str.replace(r'\s*with a story:s*$', '', case=False, regex=True)

In [367]:
# Remove "with a video:" from 'content' column after trimming and converting to lowercase   
scraped_cleaned_df['content'] = scraped_cleaned_df['content'].str.strip().str.replace(r'\s*with a video:s*$', '', case=False, regex=True)

In [368]:
# Remove "with a photo:" from 'content' column after trimming and converting to lowercase   
scraped_cleaned_df['content'] = scraped_cleaned_df['content'].str.strip().str.replace(r'\s*with a Photo:s*$', '', case=False, regex=True)

In [369]:
# Remove "attachments:" from 'content' column after trimming and converting to lowercase   
scraped_cleaned_df['content'] = scraped_cleaned_df['content'].str.strip().str.replace(r'\s*Attachments:s*$', '', case=False, regex=True)

In [370]:
scraped_cleaned_df.head()

,original,content
0,Uno sportivo estremo americano di 33 anni è mo...,Un base jumper di 33 anni si è schiantato al s...
1,L'alveare del ministro Lollobrigida è stato at...,Sterminate le api sul tetto del ministero dell...
2,"In Italia, le giovani donne con una laurea gua...",Le laureate italiane guadagnano la metà dei co...
3,"""Le opposizioni sono indisponibili a rinnovare...",Le opposizioni sono indisponibili a rinnovare ...
4,Beyoncé è troppo nera per i Country Music Awar...,"Beyoncé 'troppo nera', snobbata ai premi del c..."


In [373]:
# Replace empty strings with pd.NA
scraped_cleaned_df['original'].str.strip()
scraped_cleaned_df['content'].str.strip()
scraped_cleaned_df.replace("", pd.NA, inplace=True)

# Remove rows with NA
scraped_cleaned_df = scraped_cleaned_df.dropna()

In [375]:
scraped_cleaned_df.head()

,original,content
0,Uno sportivo estremo americano di 33 anni è mo...,Un base jumper di 33 anni si è schiantato al s...
1,L'alveare del ministro Lollobrigida è stato at...,Sterminate le api sul tetto del ministero dell...
2,"In Italia, le giovani donne con una laurea gua...",Le laureate italiane guadagnano la metà dei co...
3,"""Le opposizioni sono indisponibili a rinnovare...",Le opposizioni sono indisponibili a rinnovare ...
4,Beyoncé è troppo nera per i Country Music Awar...,"Beyoncé 'troppo nera', snobbata ai premi del c..."


### Saving the dataset

In [380]:
# Rename the columns
scraped_cleaned_df = scraped_cleaned_df.rename(columns={'original': 'journalistic', 'content': 'conversational'})

In [381]:
scraped_cleaned_df.head()

,journalistic,conversational
0,Uno sportivo estremo americano di 33 anni è mo...,Un base jumper di 33 anni si è schiantato al s...
1,L'alveare del ministro Lollobrigida è stato at...,Sterminate le api sul tetto del ministero dell...
2,"In Italia, le giovani donne con una laurea gua...",Le laureate italiane guadagnano la metà dei co...
3,"""Le opposizioni sono indisponibili a rinnovare...",Le opposizioni sono indisponibili a rinnovare ...
4,Beyoncé è troppo nera per i Country Music Awar...,"Beyoncé 'troppo nera', snobbata ai premi del c..."


In [384]:
scraped_cleaned_df.to_csv("../corpora/J2C_news_IT.csv", index=False)